<a href="https://colab.research.google.com/github/MuhammadShavaiz/AI_learning/blob/main/STT%2BSummarization_Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import subprocess
import sys

# List of dependencies
dependencies = [
    "torch",
    "pyaudio",
    "numpy",
    "scipy",
    "transformers"
]

# Function to install dependencies
def install_dependencies(dependencies):
    for package in dependencies:
        try:
            print(f"Installing {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print(f"{package} installed successfully.\n")
        except subprocess.CalledProcessError:
            print(f"Failed to install {package}. Please try installing it manually.")

if __name__ == "__main__":
    install_dependencies(dependencies)


Installing torch...
torch installed successfully.

Installing pyaudio...
Failed to install pyaudio. Please try installing it manually.
Installing numpy...
numpy installed successfully.

Installing scipy...
scipy installed successfully.

Installing transformers...
transformers installed successfully.



In [20]:
from    transformers            import  WhisperProcessor, WhisperForConditionalGeneration

def openai_summarizer(index = 3):
    """
    https://huggingface.co/openai/whisper-tiny
    """
    modelss =   [
                "openai/whisper-tiny",      #39m params useless
                "openai/whisper-base",      #74m params
                "openai/whisper-small",     #244m params very well
                "openai/whisper-medium",    #769m params
                "openai/whisper-large",     #1550m params
                "openai/whisper-large-v2",  #1550m params
                ]

    # load model and processor
    processor   = WhisperProcessor.from_pretrained(modelss[index])
    model       = WhisperForConditionalGeneration.from_pretrained(modelss[index]).to('cuda')
    model.config.forced_decoder_ids = None

    return processor,model

In [21]:
import wave
import numpy as np

def save_wave_file(cached_audio,
                   p,
                   CHANNELS,
                   FORMAT,
                   RATE,
                   AMPLIFICATION_FACTOR=10, output_file="cached_audio.wav"):
    """
    Save the cached audio data to a WAV file.

    Args:
        cached_audio (numpy.ndarray): The audio data to be saved.
        p (pyaudio.PyAudio): PyAudio instance.
        AMPLIFICATION_FACTOR (int, optional): Amplification factor for the audio. Default is 10.
        output_file (str, optional): Output file path. Default is "cached_audio.wav".
    """
    # Save the cached audio to a WAV file
    with wave.open(output_file, "wb") as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes((cached_audio * AMPLIFICATION_FACTOR).tobytes())
    print(f"Saved cached audio to {output_file}")


def record_audio(pyaudio,
                 CHANNELS,
                 FORMAT,
                 RATE,
                 CHUNK,
                 ):
    """
    Record audio from the microphone.

    Returns:
        tuple: Numpy array of recorded audio and PyAudio instance.
    """
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    # Initialize an empty list to store audio data
    audio_cache = []
    try:
        print("Recording... (Press Ctrl+C to stop)")
        while True:
            # Read audio data from the microphone
            data = stream.read(CHUNK)

            # Convert the binary audio data to a numpy array
            audio_sample = np.frombuffer(data, dtype=np.int16)

            # Append the audio sample to the cache
            audio_cache.append(audio_sample)

    except KeyboardInterrupt:
        print("Recording stopped.")

    # Close the audio stream and PyAudio instance
    stream.stop_stream()
    stream.close()
    p.terminate()

    # Convert the list of audio samples into a numpy array
    return np.concatenate(audio_cache), p


In [22]:
import subprocess

def encode_to_x265(input_file, output_file):
    """
    Encode a video file to x265 format using FFmpeg.

    :param input_file: Input video file path.
    :param output_file: Output file path for the encoded video.
    """
    # Use FFmpeg with libx265 to encode the video file
    ffmpeg_cmd = f'ffmpeg -i  "{input_file}" -vn -acodec pcm_s16le -ar 16000 -ac 1 "{output_file}"'
    subprocess.run(ffmpeg_cmd, shell=True)

def write_text_to_file(filename, text):
    """
    Write a text string to a file.

    :param filename: The name of the file to write to.
    :param text: The text string to write to the file.
    """
    try:
        with open(filename, 'w') as file:
            file.write(text)
        print(f"Text has been written to {filename}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")


# 1.Audio Extact

if tou want to use the bash command

```ffmpeg -i .input_file.mp4 -vn -acodec pcm_s16le -ar 16000 -ac 1  ./output.wav```

# 2.Audio-to-text

models description :

https://huggingface.co/openai/whisper-large-v2

https://huggingface.co/jonatasgrosman/wav2vec2-large-xlsr-53-english



In [23]:
processor,model = openai_summarizer()

In [41]:
from scipy.io import wavfile
import torch
import numpy as np
import librosa

# Read the audio file
samplerat1, data1 = wavfile.read('output.wav')
print("Audio data shape:", data1.shape)
print("Audio data type:", type(data1))
print("Sample rate:", samplerat1)

# Normalize and convert to float32 if the data is in int16 format
if np.issubdtype(data1.dtype, np.integer):
    data1 = data1.astype(np.float32) / np.iinfo(data1.dtype).max

# Resample the audio to 16,000 Hz
data1_resampled = librosa.resample(data1, orig_sr=samplerat1, target_sr=16000)
print("Resampled audio shape:", data1_resampled.shape)

# Check if CUDA is available and move the model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("CUDA available:", torch.cuda.is_available())

# Process the audio in chunks
chunk_size = 25 * 16000  # Length of each chunk in samples
text = ""

for index in range(0, len(data1_resampled) - chunk_size + 1, chunk_size):
    input_features = processor(
        data1_resampled[index:index + chunk_size],
        sampling_rate=16000,
        return_tensors="pt"
    ).input_features.to(device)

    predicted_ids = model.generate(input_features)
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

    print(f"Chunk {index // chunk_size + 1} transcription:", transcription)

    if transcription and transcription[0].strip():  # Check for valid transcription
        text += transcription[0] + ' '

# Print final transcription
print("Final Transcription:", text)


Audio data shape: (466063,)
Audio data type: <class 'numpy.ndarray'>
Sample rate: 8000
Resampled audio shape: (932126,)
CUDA available: True
Chunk 1 transcription: [' The horse trotted around the field at a brisk pace. Find the twin who stole the pearl necklace. Cut the cord that binds the box tightly. The red tape bound the smuggled food. Look in the corner to find']
Chunk 2 transcription: [' the tan shirt. The cold drizzle will halt the bond drive. Nine men were hired to dig the ruins. The junkyard had a moldy smell. The flint sputtered and lit a pine torch.']
Final Transcription:  The horse trotted around the field at a brisk pace. Find the twin who stole the pearl necklace. Cut the cord that binds the box tightly. The red tape bound the smuggled food. Look in the corner to find  the tan shirt. The cold drizzle will halt the bond drive. Nine men were hired to dig the ruins. The junkyard had a moldy smell. The flint sputtered and lit a pine torch. 


In [42]:
total_lenght = len(text)
' '.join(text.split(' ')[0:(total_lenght//3)])

' The horse trotted around the field at a brisk pace. Find the twin who stole the pearl necklace. Cut the cord that binds the box tightly. The red tape bound the smuggled food. Look in the corner to find  the tan shirt. The cold drizzle will halt the bond drive. Nine men were hired to dig the ruins. The junkyard had a moldy smell. The flint sputtered and lit a pine torch. '

In [43]:
import re
# Define the regular expression pattern
paragraph_pattern = r'\.'

# Your long transcript
transcript = text

# Split the transcript into paragraphs
paragraphs = re.split(paragraph_pattern, transcript)

# Optionally, strip leading/trailing whitespace
paragraphs = [p.strip() for p in paragraphs]


# Using a list comprehension to remove empty strings
filtered_list = [item for item in paragraphs if item.strip() != ""]

print(filtered_list)


['The horse trotted around the field at a brisk pace', 'Find the twin who stole the pearl necklace', 'Cut the cord that binds the box tightly', 'The red tape bound the smuggled food', 'Look in the corner to find  the tan shirt', 'The cold drizzle will halt the bond drive', 'Nine men were hired to dig the ruins', 'The junkyard had a moldy smell', 'The flint sputtered and lit a pine torch']


# 3.Summurazation

In [46]:
from transformers import pipeline
summarizer = pipeline("summarization", model="t5-small")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [50]:
print(summarizer(' '.join(filtered_list[0:(len(filtered_list)//3)]) , min_length=90, do_sample=True))


Your min_length=30 must be inferior than your max_length=15.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1399: UserWarning: Unfeasible length constraints: `min_length` (30) is larger than the maximum possible length (15). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


[{'summary_text': 'the horse trotted around the field at a brisk'}]
